In [43]:
from math import *
import pandas as pd
import numpy as np
import statistics as st
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import lagrange

#ML packages
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

## Random Forest for Determining Engine Types of Missing N-Numbers

In [17]:
flight_data = pd.read_csv('all_CA_airports_2019.csv')
aircraft_model_names = pd.read_csv('aircraft_model_names.csv')
aircraft_model_names = aircraft_model_names.rename(columns = {'Unnamed: 0': 'TailNumber'})


In [45]:
def od_pair_name_addition(df):
    a = df[['Departure', 'Arrival']].values
    a = np.sort(a)
    a_df = pd.DataFrame(a)
    a_df = pd.Series(a_df[0] + a_df[1])
    insert = pd.concat([a_df, pd.Series(df.index)], axis = 1).set_index(1)
    df_return = pd.concat([insert, df], axis = 1).rename(columns = {0: 'OD Pair Name'})
    return df_return

In [44]:
dfv1 = flight_data.merge(aircraft_model_names, left_on = 'TailNumber', right_on = 'TailNumber', how = 'left')
dfv1 = dfv1.drop('ENG MFR MDL', axis = 1);
missing = dfv1[(dfv1['MODEL'].isnull()) & ~(dfv1['AircraftType'].isnull())] #only engine model is missing; aircraft type is not missing
not_missing = dfv1[~(dfv1['AircraftType'].isnull()) & ~(dfv1['MODEL'].isnull())] #both engine model and ac type are not misisng 
not_missing['MODEL'] = not_missing['MODEL'].str.strip()
missing.index = missing.index.astype(int); not_missing.index = not_missing.index.astype(int)

In [48]:
not_missing = od_pair_name_addition(not_missing)

In [49]:
cat_features = not_missing[['OD Pair Name', 'AircraftType']]
feature_one_hot = pd.get_dummies(data = cat_features, drop_first = True)

In [51]:
X = pd.concat([not_missing['ActualAirborneTime'], feature_one_hot], axis = 1)
y = not_missing['MODEL']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
model = RandomForestClassifier(n_estimators = 100, oob_score = True, n_jobs = -1, random_state = 50, 
                              max_features = 190, min_samples_leaf = 15, verbose = 2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))